Run in public colab: https://colab.sandbox.google.com/ (not https://colab.corp.google.com/). Should work in both python 2 and 3 and also in Cloud AI platform
notebooks.


In [ ]:
try:
  from google.colab import auth
  auth.authenticate_user()  
except ImportError:
  pass

In [ ]:
from google.cloud import _http
from google.cloud.client import ClientWithProject
from xml.etree import ElementTree as ET
from IPython.display import HTML
import pprint

In [ ]:
class Connection(_http.JSONConnection):
  """Handles HTTP requests to GCP."""
  API_BASE_URL = 'https://healthcare.googleapis.com'
  API_VERSION = 'v1beta1'
  API_URL_TEMPLATE = '{api_base_url}/{api_version}/projects{path}'


class Client(ClientWithProject):
  """A client for accessing Cloud Healthcare NLP API.

  Args:
      project (Union[str, None]): The ID of the project
      region (str): The region the project resides in, e.g. us-central1,
  """

  def __init__(self,
               project=None,
               region='us-central1',
               credentials=None,
               http=None):
    self.region = region
    self.SCOPE = ('https://www.googleapis.com/auth/cloud-healthcare',)
    super(Client, self).__init__(project=project)
    self.path = '/{}/locations/{}/services/nlp'.format(self.project,
                                                       self.region)
    self._connection = Connection(self)

  def analyze_entities(self, document):
    """ Analyze the clinical entities a document with the Google Cloud

      Healthcare NLP API.

      Args:
        document (str): the medical document to analyze.

      Returns:
        Dict[str, Any]: the JSON response.
      """
    return self._connection.api_request(
        'POST',
        self.path + ':analyzeEntities',
        data={'document_content': document})

In [ ]:
# @title String fields
project_id = ''  #@param {type:"string"}

In [ ]:
client = Client(project=project_id)

In [ ]:
# @title String fields
document = 'H. pylori DNA Analysis 3 Diagnostic Solutions laboratory 877-485-5336 Patient: Ima Sample Collected: 2/10/2018 DOB: 7/11/1981 Accession: 20180212-0001 Received: 2/12/2018 Completed: 2/12/2018 Ordered by: Diane Farhi, MD H. pylori Result Normal 2.9e3 High <1.0 e3 Helicobacter pylori Virulence Factor , babA Positive Positive Virulence Factor , cagA Virulence Factor , dupa Virulence Factor, iceA Negative Negative Negative Negative Negative Negative Negative Negative Negative Negative Negative Negative Positive Positive Virulence Factor , oipA Virulence Factor , vacA Virulence Factor , virB Virulence Factor , virD Antibiotic Resistance Genes, phenotypes Helicobacter Result Clarithromycin Positive Expected Result Absent A21420 Absent A2142G Absent A2143G Present Fluoroquinolones Negative Absent gyrA N87K Absent gyrA D91N Absent gyrA D91G Absent Absent Absent gyrB S479N Tetracycline gyrB R484K Positive Absent PBP1A S414R Present PBP1A T556S Absent PBP1A N562Y Absent Amoxicillin Negative Absent A926G Absent AGA926-928TTC Absent The assays were developed and the perfomance characteristics determined by Diagnostic Solutions Laboratory. CLIA# 11D-2097795 Medical Director - Diane Farhi, MD 1'

In [ ]:
response = client.analyze_entities(document)

In [ ]:
def draw_response(doc, resp):
  """Creates an HTML summary of the AnalyzeEntities response.

    Args:
        doc (str): the document that was analyzed.
        resp (Dict[str, Any]): the JSON response.
    Return:
        (str): An HTML summary of the response
  """
  html = ET.Element('html')
  ET.SubElement(
      html, 'style').text = ('.entity_span { color: blue; font-weight: bold; }')
  body = ET.SubElement(html, 'body')

  # Format entity mentions (including linked entities in hover-text)
  start = 0
  id_to_text = {}
  for entity_mention in resp.get('entityMentions', []):
    begin = entity_mention['text'].get('beginOffset', 0)
    text = entity_mention['text'].get('content', '')
    id_to_text[entity_mention['mentionId']] = text
    ET.SubElement(body, 'span').text = doc[start:begin]
    ET.SubElement(
        body,
        'span',
        attrib={
            'class': 'entity_span',
            'title': pprint.pformat(entity_mention).replace("'", '')
        }).text = text
    start = begin + len(text)
  ET.SubElement(body, 'span').text = document[start:]

  # Format relationships
  ET.SubElement(body, 'hr')
  rel_list = ET.SubElement(body, 'ul')
  for rel in sorted(resp.get('relationships', []),
                    key = lambda i: (int(i['subjectId']),int(i['objectId']))):
    subj_id = rel['subjectId']
    obj_id = rel['objectId']
    conf = rel['confidence']
    ET.SubElement(
        rel_list,
        'li').text = ('({}) {} -> ({}) {} [confidence = {:.2f}]'.format(
            subj_id, id_to_text[subj_id], obj_id, id_to_text[obj_id], conf))

  return ET.tostring(html, 'utf-8').decode()

In [ ]:
HTML(draw_response(document, response))

# New Section

In [ ]:
pprint.pprint(response)

NameError: ignored